In [1]:
import torch
import numpy as np
from torch.distributions import Normal
import copy

In [3]:
class RQK:
    def __init__(self, length_scale=1, alpha=1, l_bounds=[-2, 2], a_bounds=[-2, 2]):
        self.length_scale = length_scale
        self.alpha = alpha
        self.l_bounds = l_bounds
        self.a_bounds = a_bounds
    
    def _params_grid(self):
        lb, ub = self.a_bounds
        a_sample = torch.logspace(lb, ub, steps=20)
        lb, ub = self.l_bounds
        l_sample = torch.logspace(lb, ub, steps=20)
        l_grid, a_grid = torch.meshgrid(l_sample, a_sample)
        params = list(zip(l_grid.ravel(), a_grid.ravel()))
        return params
    
    def __call__(self, X1, X2):
        dist = torch.cdist(X1, X2) ** 2
        dist = 1 + dist / (2 * self.alpha * self.length_scale**2)
        kernel = dist ** (-self.alpha)
        return kernel 

In [4]:
class GaussianProcess:
    def __init__(self, kernel, eps=1e-10):
        self.kernel = kernel
        self.eps = eps
    
    def fit(self, X, y):
        self.y_mean = y.mean()
        self.y_std = y.std()
        y = (y - self.y_mean) / self.y_std
        self.X_ = X.clone()
        self.y_ = y.clone()
        
        K = self.kernel(X, X)
        K[np.diag_indices(len(X))] += self.eps
        self.L = torch.linalg.cholesky(K)
        self.alpha_ = torch.cholesky_solve(y.unsqueeze(1), self.L)
    
    def predict(self, X):
        K_t = self.kernel(X, self.X_)
        y_mean = K_t.mm(self.alpha_).squeeze() * self.y_std + self.y_mean
        v = torch.triangular_solve(K_t.T, self.L, upper=False)[0]
        y_cov = self.kernel(X, X) - v.T.mm(v)
        y_cov = y_cov * self.y_std
        y_std = torch.sqrt(y_cov.diag())
        return y_mean, y_cov, y_std
    
    def log_marginal_likelihood(self, X, y):
        y = (y - y.mean()) / y.std()
        K = self.kernel(X, X)
        K[np.diag_indices_from(K)] += self.eps
        L = torch.linalg.cholesky(K)
        y = y.unsqueeze(1)
        alpha = torch.cholesky_solve(y, L)
        log_likelihood = -0.5 * y.T.mm(alpha) - torch.log(torch.diag(L)).sum() - K.shape[0] / 2 * torch.log(torch.Tensor([2*torch.pi]))
        return log_likelihood

In [5]:
def fit_gaussian_process(X, y):
    kernel = RQK()
    params = kernel._params_grid()
    mll_list = []
    for l, a in params:
        gp = GaussianProcess(RQK(l, a))
        try:
            mll = gp.log_marginal_likelihood(X, y)
            mll_list.append(mll.item())
        except:
            mll_list.append(np.nan)
    max_idx = np.nanargmax(np.array(mll_list))
    l, a = params[max_idx]

    gp = GaussianProcess(RQK(l, a))
    gp.fit(X, y)
    return gp

In [6]:
def expected_improvement(gp, X, best):
    m, cov, _ = gp.predict(X)
    std = cov.diag().clamp_min(1e-9).sqrt()
    u = (m - best) / std
    normal = Normal(torch.zeros_like(u), torch.ones_like(u))
    ucdf = normal.cdf(u)
    updf = torch.exp(normal.log_prob(u))
    ei = std * (updf  + u * ucdf)
    return ei

In [7]:
torch.manual_seed(0)
def func(x):
    return torch.sin(x[0]) / x[0] + torch.sin(x[1])/x[1] - 0.1*torch.randn(1)

In [9]:
X_list = [torch.Tensor(2).uniform_(-10, 10) for i in range(15)]
Y_list = [func(x) for x in X_list]

In [ ]:
for i in range(30):
    X = torch.stack(X_list)
    Y = torch.cat(Y_list)
    gp = fit_gaussian_process(X, Y)


    p1 = torch.linspace(-10, 10, 50)
    p2 = torch.linspace(-10, 10, 50)
    p1_grid, p2_grid = torch.meshgrid(p1, p2)
    X_candidate = torch.stack([p1_grid.ravel(), p2_grid.ravel()]).T
    
    best = Y.max().item()
    ei = expected_improvement(gp, X_candidate, best)
    X_next = X_candidate[ei.argmax()]
    Y_next = func(X_next)
    X_list.append(X_next)
    Y_list.append(Y_next)